In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9113847460950942752
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11324823962
locality {
  bus_id: 1
}
incarnation: 10912970235895781555
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from helpers import *
import os, sys, tarfile, time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.ticker import MaxNLocator

params = {'legend.fontsize': 'x-large',
          'font.size'      : 10,
          'figure.figsize' : (17, 3),
          'axes.labelsize' : 'x-large',
          'axes.titlesize' : 'x-large',
          'axes.grid'      : 'on',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}

pylab.rcParams.update(params)

RATINGS_PATH = './data_ml-20m/ratings.csv'
MOVIES_PATH = './data_ml-20m/movies.csv'

#...................................................
#...................................................

with open(RATINGS_PATH) as f:
    ratings = f.readlines()

with open(MOVIES_PATH) as f:
    movies = f.readlines()

type(ratings), sys.getsizeof(ratings)/1e6, len(ratings)/1e6

#...................................................
#...................................................

from_mID_2_idx = {}
for i, row in enumerate(movies[1:]):
    from_mID_2_idx[int(row.split(',')[0])] = i
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
n_users = 138493 - 1
n_movies = len(movies) - 1

start = time.time()
Y_np = np.zeros((n_users,n_movies), dtype=np.float32)
R_np = np.zeros((n_users,n_movies), dtype=np.int8)
for i, row in enumerate(ratings[1:]):
    uID, mID, yij = [e for e in row.split(',')[:-1]]
    yij = float(yij)
    u_idx = int(row[0]) - 1
    m_idx = from_mID_2_idx[int(mID)]

    Y_np[u_idx,m_idx] = yij
    R_np[u_idx,m_idx] = 1
    
print_runtime(start)

# npy_saver(Y,'Y')
# npy_saver(R,'R')

# Y = npy_loader('Y')
# R = npy_loader('R')

Runtime: 0 min 56 sec


In [22]:
Y = tf.placeholder(dtype=tf.float32, shape=(None,n_movies))
R = tf.placeholder(dtype=tf.float32, shape=(None,n_movies))

k = 100
U = tf.Variable(tf.truncated_normal(shape=(n_users,k)),dtype=tf.float32)
V = tf.Variable(tf.truncated_normal(shape=(k,n_movies)),dtype=tf.float32)

Y_pred = tf.reduce_sum(tf.matmul(U, V))
loss = tf.norm(Y - tf.multiply(R, Y_pred))

lr = .01
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train = optimizer.minimize(loss=loss)

In [18]:
batch_Y.shape

(128, 27278)

In [19]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
b_idx = 0
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for step in range(NUM_EPOCHS):
        batch_Y, batch_R = next_batch(Y_np, R_np, BATCH_SIZE)
        train_acc = sess.run(train, feed_dict={Y:batch_Y, R:batch_R})
        print("step {}, trainin_acc {}".format(step,train_acc))
        

ResourceExhaustedError: OOM when allocating tensor with shape[138492,27278]
	 [[Node: MatMul_2 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_4/read, transpose_2)]]

Caused by op 'MatMul_2', defined at:
  File "/home/aind2/anaconda3/envs/aind-dog/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/aind2/anaconda3/envs/aind-dog/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/aind2/anaconda3/envs/aind-dog/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aind2/anaconda3/envs/aind-dog/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aind2/anaconda3/envs/aind-dog/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/aind2/anaconda3

In [21]:
def wrapper(Y_np, R_np, batch_idx):
    batch_idx = 0
    def next_batch(Y_np, R_np, batch_idx):
        batch_idx += BATCH_SIZE
        i0 = batch_idx
        i1 = min(batch_idx + BATCH_SIZE, len(Y)-1)
        
        yield batch_Y[i0:i1], batch_R[i0:i1] 
    